In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# This folder should contain a set of folders with the output of training a model in SLEAP
model_folder = "../models/myee1"

In [3]:
[file for file in os.scandir(model_folder) if file.is_dir()]

[<DirEntry '240417_192635.centroid.n=80'>,
 <DirEntry '240417_211609.centered_instance.n=80'>,
 <DirEntry 'grayscale.240415_124620.centroid'>,
 <DirEntry 'half_size.240416_202503.multi_instance'>,
 <DirEntry 'rgb.240415_124620.centroid'>,
 <DirEntry 'rgb.240416_121034.multi_instance'>]

In [33]:
summary_stats = ['vis.tp',
                 'vis.fp',
                 'vis.tn',
                 'vis.fn',
                 'vis.precision',
                 'vis.recall',
                 'dist.avg',
                 'dist.p50',
                 'dist.p75',
                 'dist.p90',
                 'dist.p95',
                 'dist.p99',
                 'oks_voc.mAP',
                 'oks_voc.mAR',
                 'pck_voc.mAP',
                 'pck_voc.mAR']

all_stats = []

for entry in os.scandir(model_folder):
    if entry.is_dir():
        for file in os.scandir(entry):
            if file.is_file() and file.name.endswith(".npz"):
                with np.load(file.path, None, True) as np_file:
                    #display(np_file['metrics'][()].keys())
                    stats = {'model': entry.name, 'filename': file.name}
                    stats.update({key: np_file['metrics'][()][key][()] for key in np_file['metrics'][()] if key in summary_stats})
                    all_stats.append(stats)
                    #display(pd.DataFrame(stats, index=[entry.name, file.name]))

pd.DataFrame(all_stats)

,model,filename,vis.tp,vis.fp,vis.tn,vis.fn,vis.precision,vis.recall,dist.avg,dist.p50,dist.p75,dist.p90,dist.p95,dist.p99,oks_voc.mAP,oks_voc.mAR,pck_voc.mAP,pck_voc.mAR
0,240417_192635.centroid.n=80,metrics.train.npz,24370,200,0,0,0.991860,1.000000,46.834515,0.000067,0.000108,229.446071,356.146906,540.869825,0.739381,0.796103,0.739381,0.796103
1,240417_192635.centroid.n=80,metrics.val.npz,2175,21,0,0,0.990437,1.000000,107.438535,0.000087,223.759914,418.149826,499.738397,635.418351,0.533383,0.583333,0.533383,0.583333
2,240417_211609.centered_instance.n=80,metrics.train.npz,25314,119,174,187,0.995321,0.992667,4.845595,2.060915,3.088830,4.682363,6.792886,118.642727,0.846243,0.907816,0.542564,0.612142
3,240417_211609.centered_instance.n=80,metrics.val.npz,2574,27,9,54,0.989619,0.979452,10.244148,2.438795,4.196757,9.059024,54.202832,175.325526,0.666528,0.745270,0.346497,0.427703
4,grayscale.240415_124620.centroid,metrics.train.npz,15516,144,0,0,0.990805,1.000000,64.344455,0.000070,0.000115,306.492998,411.516963,596.547072,0.682251,0.736451,0.682251,0.736451
5,grayscale.240415_124620.centroid,metrics.val.npz,3543,39,0,0,0.989112,1.000000,60.687330,0.000073,0.000119,274.798020,361.707586,536.300614,0.626528,0.687500,0.626528,0.687500
6,half_size.240416_202503.multi_instance,metrics.train.npz,21309,207,57,279,0.990379,0.987076,15.625757,6.739177,12.972711,24.013707,36.870252,264.251308,0.271087,0.347343,0.002683,0.013246
7,half_size.240416_202503.multi_instance,metrics.val.npz,5649,43,9,347,0.992446,0.942128,85.587394,9.437929,88.383654,336.229756,426.404389,577.782761,0.169746,0.246927,0.002308,0.009497
8,rgb.240415_124620.centroid,metrics.train.npz,14952,132,0,0,0.991249,1.000000,61.848762,0.000069,0.000116,297.197873,398.107385,545.639094,0.653179,0.707758,0.653179,0.707758
9,rgb.240415_124620.centroid,metrics.val.npz,3620,34,0,0,0.990695,1.000000,67.636892,0.000071,0.000116,317.400110,417.460153,626.215737,0.664553,0.718750,0.664553,0.718750
